In [1]:
import osmnx as ox
import random
import lib
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

In [2]:
G = lib.load()
allHouses, allInfra = lib.getTaggedNodes(G)

In [60]:
# 1.1 params
speed = 40 # km/h

houses = random.sample(allHouses, 100)
infra = random.sample(allInfra, 10)

maxTime = 20
maxDistance = 2000

In [61]:
progress = IntProgress(min=0, max=len(houses) + len(infra), value=0)
display(progress)

def updateProgress():
    progress.value += 1
    progress.description = f'{progress.value}/{progress.max}'

houseObjPaths = lib.getManyToManyPaths(G, houses, infra, speed, updateProgress)
objHousePaths = lib.getManyToManyPaths(G, infra, houses, speed, updateProgress)
roundPaths = {}

# Make round paths form them
for house in houses:
    for obj in infra:
        if(obj in houseObjPaths[house]) and (house in objHousePaths[obj]):
            roundPaths[f"{house}-{obj}"] = lib.joinPaths(houseObjPaths[house][obj], objHousePaths[obj][house])
            roundPaths[f"{obj}-{house}"] = lib.joinPaths(objHousePaths[obj][house], houseObjPaths[house][obj])
        
progress.bar_style = "success" 

IntProgress(value=0, max=110)

In [62]:
houseClosestObj = {}
for house in houseObjPaths:
    shortestPath = {"length": float('inf')}
    closestObj = None
    for obj in houseObjPaths[house]:
        if(houseObjPaths[house][obj]['length'] < shortestPath['length']):
            shortestPath = houseObjPaths[house][obj]
            closestObj = obj
    
    houseClosestObj[house] = closestObj

In [63]:
objClosestHouse = {}
for obj in infra:
    shortestPath = {"length": float('inf')}
    closestHouse = None
    for house in objHousePaths[obj]:
        if(objHousePaths[obj][house]['length'] < shortestPath['length']):
            shortestPath = objHousePaths[obj][house]
            closestHouse = house
    
    objClosestHouse[obj] = closestHouse

In [64]:
shortestHouseObjRounds = {}
for house in houseObjPaths:
    shortestPath = {"length": float('inf')}
    closestObj = None
    for obj in houseObjPaths[house]:
        if (f'{house}-{obj}' in roundPaths) and (roundPaths[f'{house}-{obj}']['length'] < shortestPath['length']):
            shortestPath = roundPaths[f'{house}-{obj}']
            closestObj = obj
    
    shortestHouseObjRounds[house] = closestObj

In [82]:
print("1.a От дома")
houseClosestObj

1.a От узла


{48897900: 45980009,
 40725974: 653881265,
 40723917: 653881265,
 46752189: 363303438,
 71373786: 60446241,
 48897904: 45980009,
 79190781: 392202653,
 66823614: 392202653,
 67480011: 392202653,
 35223064: 363303438,
 66823619: 392202653,
 88816676: 363303438,
 45898724: 45980009,
 6155998: 392202653,
 35284507: 45980009,
 295954933: 60446241,
 61358326: 60446241,
 49188929: 81236360,
 48908680: 172243520,
 50280268: 363303438,
 40187592: 172243520,
 48538675: 81236360,
 62104592: 172243520,
 90997758: 81236360,
 751825226: 38353503,
 55227804: 363303438,
 49250821: 81236360,
 58217775: 81146143,
 96703443: 81146143,
 125270674: 392202653,
 94927974: 60446241,
 66823537: 392202653,
 81146079: 81146143,
 246088420: 363303438,
 50280318: 363303438,
 529027502: 363303438,
 105727180: 392202653,
 60446325: 60446241,
 77332375: 81236360,
 55482753: 363303438,
 66229056: 392202653,
 55962418: 363303438,
 35474866: 45980009,
 35286418: 45980009,
 62104545: 172243520,
 61512280: 60446241,
 493

In [80]:
print("1.a От объекта")
objClosestHouse

1.a От объекта


{363303438: 46752198,
 45980009: 45898724,
 81236360: 181552390,
 60446241: 94927983,
 653881265: 50874981,
 38353503: 35899999,
 392202653: 107507929,
 81146143: 81146079,
 300218821: 55482753,
 172243520: 62104592}

In [81]:
print("1.a Туда и обратно")
shortestHouseObjRounds

1.a Туда обратно


{48897900: 45980009,
 40725974: 653881265,
 40723917: 653881265,
 46752189: 363303438,
 71373786: 60446241,
 48897904: 45980009,
 79190781: 392202653,
 66823614: 392202653,
 67480011: 392202653,
 35223064: 363303438,
 66823619: 392202653,
 88816676: 363303438,
 45898724: 45980009,
 6155998: 392202653,
 35284507: 45980009,
 295954933: 60446241,
 61358326: 60446241,
 49188929: 81236360,
 48908680: 172243520,
 50280268: 363303438,
 40187592: 653881265,
 48538675: 81236360,
 62104592: 172243520,
 90997758: 81236360,
 751825226: 38353503,
 55227804: 363303438,
 49250821: 81236360,
 58217775: 81146143,
 96703443: 81146143,
 125270674: 392202653,
 94927974: 60446241,
 66823537: 392202653,
 81146079: 81146143,
 246088420: 363303438,
 50280318: 363303438,
 529027502: 363303438,
 105727180: 392202653,
 60446325: 60446241,
 77332375: 81236360,
 55482753: 363303438,
 66229056: 392202653,
 55962418: 363303438,
 35474866: 45980009,
 35286418: 45980009,
 62104545: 172243520,
 61512280: 60446241,
 493

In [77]:
def select(paths, predicate):
    result = []
    for target in paths:
        if predicate(paths[target]):
            result.append(target)
    if len(result) == 0:
        return None
    return result
            
def byTime(path):
    return path['time'] <= maxTime
    
def byDist(path):
    return path['length'] <= maxDistance
    
def mapDict(d, mapF):
    result = {}
    for key in d:
        mapped = mapF(key, d[key])
        if mapped != None:
            result[key] = mapped
    return result

def invert(d):
    result = {}
    for key in d:
        for item in d[key]:
            if not item in result:
                result[item] = []
            result[item].append(key)
    return result

def round2HouseObj(arr):
    result = {}
    
    if arr == None:
        return result

    for pair in arr:
        house, obj = map(int, pair.split('-'))
        if not house in result:
            result[house] = []
        result[house].append(obj)
    return result

houseObjByTime = mapDict(houseObjPaths,  lambda _, paths: select(paths, byTime))
houseObjByDist = mapDict(houseObjPaths,  lambda _, paths: select(paths, byDist))
                        
houseObjBackByTime = invert(mapDict(objHousePaths,  lambda _, paths: select(paths, byTime)))
houseObjBackByDist = invert(mapDict(objHousePaths,  lambda _, paths: select(paths, byDist)))

houseObjRoundByTime = round2HouseObj(select(roundPaths, byTime))
houseObjRoundByDist = round2HouseObj(select(roundPaths, byDist))

In [79]:
print(f"1.a От дома по ближе {maxTime} минут")
houseObjRoundByTime

{40725974: [60446241, 653881265, 38353503, 81146143],
 60446241: [40725974,
  40723917,
  71373786,
  295954933,
  61358326,
  48908680,
  40187592,
  62104592,
  58217775,
  94927974,
  60446325,
  62104545,
  61512280,
  31638937,
  61433462,
  205587018,
  31645138,
  50874981,
  94927983,
  61433382,
  40187384,
  35220642,
  62041006,
  38192539,
  181552390],
 653881265: [40725974,
  40723917,
  71373786,
  295954933,
  61358326,
  48908680,
  40187592,
  62104592,
  58217775,
  96703443,
  94927974,
  81146079,
  60446325,
  62104545,
  61512280,
  31638937,
  61433462,
  205587018,
  31645138,
  50874981,
  94927983,
  80584890,
  61433382,
  40187384,
  35220642,
  49578354,
  62041006,
  38192539,
  181552390],
 38353503: [40725974,
  751825226,
  58217775,
  96703443,
  81146079,
  50874981,
  80584890,
  35899999,
  40187384,
  38192539],
 81146143: [40725974,
  40723917,
  751825226,
  58217775,
  96703443,
  81146079,
  205587018,
  31645138,
  50874981,
  80584890,
  358

In [87]:
print(f"1.a От дома по ближе {maxDistance} метров")
houseObjByDist

1.a От дома по ближе 2000 метров


{46752189: [363303438, 45980009],
 45898724: [363303438, 45980009],
 295954933: [60446241],
 50280268: [363303438, 45980009],
 62104592: [172243520],
 94927974: [60446241],
 81146079: [81146143],
 246088420: [363303438, 45980009],
 50280318: [363303438],
 529027502: [363303438, 45980009],
 55482753: [363303438, 45980009, 300218821],
 45984691: [363303438, 45980009],
 50874981: [653881265],
 228234712: [363303438, 45980009],
 50344910: [363303438, 45980009],
 94927983: [60446241],
 80584890: [81146143],
 35899999: [38353503],
 107507929: [392202653],
 35220642: [653881265],
 45893507: [363303438, 45980009],
 38192539: [653881265],
 181552390: [172243520],
 46752198: [363303438, 45980009],
 52025778: [392202653]}

In [89]:
print(f"1.a От дома по ближе {maxTime} минут обратной дороги")
houseObjBackByTime

1.a От дома по ближе 20 минут обратной дороги


{48897900: [363303438, 45980009, 81236360, 392202653, 300218821],
 40725974: [363303438,
  45980009,
  81236360,
  60446241,
  653881265,
  38353503,
  81146143,
  300218821,
  172243520],
 40723917: [363303438,
  45980009,
  81236360,
  60446241,
  653881265,
  38353503,
  392202653,
  81146143,
  300218821,
  172243520],
 46752189: [363303438,
  45980009,
  81236360,
  60446241,
  653881265,
  392202653,
  81146143,
  300218821,
  172243520],
 71373786: [363303438,
  45980009,
  81236360,
  60446241,
  653881265,
  38353503,
  81146143,
  300218821,
  172243520],
 48897904: [363303438, 45980009, 81236360, 392202653, 300218821],
 66823614: [363303438,
  45980009,
  81236360,
  60446241,
  653881265,
  392202653,
  300218821,
  172243520],
 67480011: [363303438,
  45980009,
  81236360,
  60446241,
  653881265,
  392202653,
  300218821,
  172243520],
 35223064: [363303438,
  45980009,
  81236360,
  60446241,
  653881265,
  392202653,
  300218821,
  172243520],
 66823619: [363303438,
  4

In [93]:
print(f"1.a От дома по ближе {maxDistance} метров обратной дороги")
houseObjBackByDist

1.a От дома по ближе 2000 метров обратной дороги


{46752189: [363303438, 45980009, 300218821],
 45898724: [363303438, 45980009, 300218821],
 50280268: [363303438, 45980009],
 246088420: [363303438, 45980009],
 50280318: [363303438],
 529027502: [363303438, 45980009],
 55482753: [363303438, 45980009, 300218821],
 45984691: [363303438, 45980009],
 228234712: [363303438, 45980009],
 50344910: [363303438, 45980009],
 45893507: [363303438, 45980009, 300218821],
 46752198: [363303438, 45980009, 300218821],
 295954933: [60446241],
 94927974: [60446241],
 94927983: [60446241],
 50874981: [653881265],
 35220642: [653881265],
 38192539: [653881265],
 35899999: [38353503],
 81146079: [81146143],
 80584890: [81146143],
 62104592: [172243520],
 181552390: [172243520]}

In [91]:
print(f"1.a От дома по ближе {maxTime} минут туда-обратно")
houseObjRoundByTime

1.a От дома по ближе 20 минут туда-обратно


{40725974: [60446241, 653881265, 38353503, 81146143],
 60446241: [40725974,
  40723917,
  71373786,
  295954933,
  61358326,
  48908680,
  40187592,
  62104592,
  58217775,
  94927974,
  60446325,
  62104545,
  61512280,
  31638937,
  61433462,
  205587018,
  31645138,
  50874981,
  94927983,
  61433382,
  40187384,
  35220642,
  62041006,
  38192539,
  181552390],
 653881265: [40725974,
  40723917,
  71373786,
  295954933,
  61358326,
  48908680,
  40187592,
  62104592,
  58217775,
  96703443,
  94927974,
  81146079,
  60446325,
  62104545,
  61512280,
  31638937,
  61433462,
  205587018,
  31645138,
  50874981,
  94927983,
  80584890,
  61433382,
  40187384,
  35220642,
  49578354,
  62041006,
  38192539,
  181552390],
 38353503: [40725974,
  751825226,
  58217775,
  96703443,
  81146079,
  50874981,
  80584890,
  35899999,
  40187384,
  38192539],
 81146143: [40725974,
  40723917,
  751825226,
  58217775,
  96703443,
  81146079,
  205587018,
  31645138,
  50874981,
  80584890,
  358

In [92]:
print(f"1.a От дома по ближе {maxDistance} метров туда-обратно")
houseObjRoundByDist

1.a От дома по ближе 2000 метров туда-обратно


{46752189: [363303438],
 363303438: [46752189, 50280268, 246088420, 529027502, 55482753, 46752198],
 45898724: [45980009],
 45980009: [45898724, 45893507],
 50280268: [363303438],
 81146079: [81146143],
 81146143: [81146079, 80584890],
 246088420: [363303438],
 529027502: [363303438],
 55482753: [363303438],
 50874981: [653881265],
 653881265: [50874981],
 80584890: [81146143],
 45893507: [45980009],
 46752198: [363303438]}

In [119]:
objMax = {}

for obj in infra:
    toMax = -1
    toHouse = None
    
    fromMax = -1
    fromHouse = None
        
    roundMax = -1
    roundHouse = None

    for house in houses:
        if houseObjPaths[house][obj]['length'] > toMax:
            toMax = houseObjPaths[house][obj]['length']
            toHouse = house
            
        if objHousePaths[obj][house]['length'] > fromMax:
            fromMax = houseObjPaths[house][obj]['length']
            fromHouse = house
            
        if roundPaths[f"{house}-{obj}"]['length'] > roundMax:
            roundMax = roundPaths[f"{house}-{obj}"]['length']
            roundHouse = house
            
    objMax[obj] = {
        "to": {
            "distance": toMax,
            "house": toHouse
        },
        "from": {
            "distance": fromMax,
            "house": fromHouse
        },
        "round": {
            "distance": roundMax,
            "house": roundHouse
        }
    }

def findMinPath(kind):
    distance = float('inf')
    minObj = None
    for obj in objMax:
        if objMax[obj][kind]["distance"] < distance:
            distance = objMax[obj][kind]["distance"]
            minObj = obj
    return distance, minObj
    
objMinMaxTo = findMinPath('to')
objMinMaxFrom = findMinPath('from')
objMinMaxRound = findMinPath('round')

In [120]:
obj = objMinMaxTo[1]
house = objMax[obj]['to']['house']

print("1.2 Объект с минимальным расстоянием от самого дальнего дома (дом -> объект)")
print("Длина:", objMinMaxTo[0])
print("Объект:", obj)
print("Путь:", houseObjPaths[house][obj])

1.2 Объект с минимальным расстоянием от самого дальнего дома (дом -> объект)
Длина: 17757.057999999997
Объект: 300218821
Путь: {'route': [93573768, 771642873, 771643251, 1085347293, 771642599, 1085348124, 1085349190, 1085349192, 1085348484, 771641809, 1085346955, 771642246, 1039314427, 771643818, 771642485, 1249784511, 771643628, 1956070018, 1088801337, 858060380, 1616452570, 2371512243, 1039315233, 858060226, 1772227852, 771642816, 771641757, 1772227864, 637035582, 2183289695, 3059131803, 3059131818, 3059131792, 3253204049, 3289992580, 3289992579, 3289992577, 474622855, 474622854, 1605994646, 474622841, 941363501, 474622839, 941363619, 3128929329, 941363503, 484756703, 1527911058, 968631536, 2280869155, 968631542, 981877849, 312978701, 1409358009, 1196573792, 1195688598, 1081919181, 2216247432, 765589671, 312978501, 312978612, 312978613, 621219536, 688358713, 621219530, 345658084, 345683398, 345581421, 300218821], 'length': 17757.057999999997, 'time': 26.635586999999997}


In [121]:
obj = objMinMaxFrom[1]
house = objMax[obj]['from']['house']

print("1.2 Объект с минимальным расстоянием от самого дальнего дома (объект -> дом)")
print("Длина:", objMinMaxFrom[0])
print("Объект:", obj)
print("Путь:", objHousePaths[obj][house])

1.2 Объект с минимальным расстоянием от самого дальнего дома (объект -> дом)
Длина: 17757.057999999997
Объект: 300218821
Путь: {'route': [300218821, 345581421, 345683378, 345683330, 698266971, 410800446, 345683335, 2010073064, 1423652305, 1434011053, 242921340, 345683351, 345658084, 621219530, 688358713, 621219536, 312978613, 312978612, 312978501, 765589671, 2216247432, 1081919181, 1195688598, 1196573792, 1409358009, 312978701, 981877849, 968631542, 1256843901, 2280869153, 5726289595, 968631536, 1527911058, 484756703, 941363488, 941363270, 941363283, 941363326, 941363322, 474622841, 1605994646, 474622854, 474622855, 3289992577, 771643908, 3253204050, 3059131793, 3253204057, 3321011246, 637035582, 1772227864, 771641757, 771642816, 1772227852, 858060226, 1039315233, 2371512243, 1616452570, 858060380, 1088801337, 1956070018, 771643628, 1249784511, 771642485, 771643818, 1039314427, 771642246, 1085346955, 771641809, 1085348484, 1085349192, 1085349190, 1085348124, 771642599, 1085347293, 7716

In [122]:
obj = objMinMaxRound[1]
house = objMax[obj]['round']['house']

print("1.2 Объект с минимальным расстоянием до самого дальнего дома (дом -> объект -> дом)")
print("Длина:", objMinMaxRound[0])
print("Объект:", obj)
print("Путь:", roundPaths[f"{house}-{obj}"])

1.2 Объект с минимальным расстоянием до самого дальнего дома (дом -> объект -> дом)
Длина: 35465.22499999999
Объект: 300218821
Путь: {'route': [93573768, 771642873, 771643251, 1085347293, 771642599, 1085348124, 1085349190, 1085349192, 1085348484, 771641809, 1085346955, 771642246, 1039314427, 771643818, 771642485, 1249784511, 771643628, 1956070018, 1088801337, 858060380, 1616452570, 2371512243, 1039315233, 858060226, 1772227852, 771642816, 771641757, 1772227864, 637035582, 2183289695, 3059131803, 3059131818, 3059131792, 3253204049, 3289992580, 3289992579, 3289992577, 474622855, 474622854, 1605994646, 474622841, 941363501, 474622839, 941363619, 3128929329, 941363503, 484756703, 1527911058, 968631536, 2280869155, 968631542, 981877849, 312978701, 1409358009, 1196573792, 1195688598, 1081919181, 2216247432, 765589671, 312978501, 312978612, 312978613, 621219536, 688358713, 621219530, 345658084, 345683398, 345581421, 300218821, 300218821, 345581421, 345683378, 345683330, 698266971, 410800446, 